# Setup

In [ ]:
!pip install torch_geometric
!pip install rdkit-pypi

In [ ]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import MoleculeNet

from torch_geometric.nn import GATv2Conv, DeepGCNLayer
from torch_geometric.nn import global_mean_pool as gap
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Lipinski, Crippen, MolSurf, Fragments

from sklearn.metrics import roc_auc_score

# Training

## Preprocessing

In [ ]:
train_dataset = torch.load("train_data.pt")
valid_dataset = torch.load("valid_data.pt")

In [ ]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

## Build model

In [ ]:
class AtomEncoder(torch.nn.Module):
    def __init__(self, num_features, hidden_channels):
        super(AtomEncoder, self).__init__()

        self.embeddings = torch.nn.ModuleList()

        for i in range(9):
            self.embeddings.append(torch.nn.Embedding(100, hidden_channels))
            
        if num_features > 9:
            self.embeddings.append(torch.nn.Embedding(10000, hidden_channels))
            self.embeddings.append(torch.nn.Embedding(200000, hidden_channels))

            for i in range(2):
                self.embeddings.append(torch.nn.Embedding(10000, hidden_channels))

            for i in range(2):
                self.embeddings.append(torch.nn.Embedding(100, hidden_channels))

    def reset_parameters(self):
        for embedding in self.embeddings:
            embedding.reset_parameters()

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(1)

        out = 0
        for i in range(x.size(1)):
            out += self.embeddings[i](x[:, i])
        return out

    
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.emb = AtomEncoder(num_node_features, hidden_channels=100)
        self.gat1 = GATv2Conv(hidden_channels, hidden_channels, heads=18)
        self.gat2 = GATv2Conv(hidden_channels*18, hidden_channels, heads=9)
        self.gat3 = GATv2Conv(hidden_channels*9, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, batch):
        x , edge_index, batch_size = batch.x, batch.edge_index, batch.batch
        x = self.emb(x)
        x = self.gat1(x, edge_index)
        x = x.relu()
        x = self.gat2(x, edge_index)
        x = x.relu()
        x = self.gat3(x, edge_index)
        x = gap(x, batch_size)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

## Train Model

In [ ]:
def train(model, device, loader, optimizer, criterion):
    model.train()

    for step, batch in tqdm(enumerate(loader), total=len(loader)):
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(pred.shape).to(torch.float64)

        optimizer.zero_grad()
        is_labeled = batch.y == batch.y
        loss = criterion(pred.to(torch.float32)[is_labeled], batch.y.to(torch.float32)[is_labeled]).mean()
        loss.backward()
        optimizer.step()

def eval(model, device, loader):
    model.eval()
    y_true = []
    y_pred = []
    
    for batch in loader:

        batch = batch.to(device)
        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape))
            y_pred.append(pred)

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    
    rocauc_list = []

    for i in range(y_true.shape[1]):
        if np.sum(y_true[:,i] == 1) > 0 and np.sum(y_true[:,i] == 0) > 0:
            is_labeled = y_true[:,i] == y_true[:,i]
            rocauc_list.append(roc_auc_score(y_true[is_labeled,i], y_pred[is_labeled,i]))

    if len(rocauc_list) == 0:
        raise RuntimeError('No positively labeled data available. Cannot compute ROC-AUC.')

    return {'rocauc': sum(rocauc_list)/len(rocauc_list)}

In [ ]:
model = GCN(100, 9, 12)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss(reduction = "none")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Start training...")
for epoch in range(10):
    print("====epoch " + str(epoch + 1))

    # training
    train(model, device, train_loader, optimizer, criterion)

    # evaluating
    train_acc = eval(model, device, train_loader)
    val_acc = eval(model, device, val_loader)
    print({'Train': train_acc, 'Validation': val_acc})

# Prediction

In [ ]:
test_dataset = torch.load("test_data.pt")
outputs = pd.DataFrame([model(data).squeeze().detach().numpy() for data in test_dataset])
outputs.to_csv("test_output.csv", header=False, index=False)

# Additional Work

## Molecule representation

In [ ]:
def calculate_molecular_descriptors(mol):
    # determine molecule-wide features using rdkit
    fr_phos = Fragments.fr_phos_acid(mol) + Fragments.fr_phos_ester(mol)
    aromatic_carbocycles = Lipinski.NumAromaticCarbocycles(mol)
    mol_log_p = int(100*Crippen.MolLogP(mol) + 4417)
    peoe_vsa1 = int(1000*MolSurf.PEOE_VSA1(mol))

    mol_weight = int(Descriptors.MolWt(mol))
    tpsa = int(Descriptors.TPSA(mol))
    num_h_donors = Lipinski.NumHDonors(mol)
    num_h_acceptors = Lipinski.NumHAcceptors(mol)

    return np.array([fr_phos, aromatic_carbocycles, mol_log_p, peoe_vsa1, mol_weight, tpsa, num_h_donors, num_h_acceptors])

def atom_features(atom, molecular_descriptors):
    # determine node features using rdkit; molecule-wide features are stored here too
    atom_feature = np.array([
        atom.GetAtomicNum(),
        atom.GetTotalValence(),
        atom.IsInRing(),
        atom.GetTotalDegree(),
        atom.GetNumRadicalElectrons(),
        atom.GetIsAromatic(),
        atom.GetHybridization().real
    ])
    return np.concatenate([atom_feature, molecular_descriptors])

def bond_features(bond):
    # determine bond features using rdkit
    return np.array([
        bond.GetBondTypeAsDouble(),
        bond.GetIsConjugated(),
        bond.GetIsAromatic(),
        bond.GetStereo(),
    ])

def generate_fingerprints(mol):
    # calculate molecular fingerprint
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
    return np.array(fp)

def molecule_to_graph(smiles, label=None):
    # load rdkit molecule from SMILES
    mol = Chem.MolFromSmiles(smiles)
    
    # calculate molecule-wide features
    molecular_descriptors = calculate_molecular_descriptors(mol)
    fingerprints = generate_fingerprints(mol)

    # calculate node features
    node_features = np.array([atom_features(atom, molecular_descriptors) for atom in mol.GetAtoms()])
    x = torch.tensor(node_features, dtype=torch.long)

    # generate edge matrix, edge features
    edge_indices = []
    edge_features = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_indices += [[i, j], [j, i]]
        edge_features += [bond_features(bond), bond_features(bond)]

    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_features, dtype=torch.long)
    fp = torch.tensor(fingerprints, dtype=torch.long)

    # if labelled, include in Data object. fingerprints stored as their own field.
    if label is not None:
        y = label
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, fp=fp)
    else:
        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, fp=fp)

In [ ]:
train_features = [molecule_to_graph(raw.smiles, raw.y) for raw in train_dataset]
valid_features = [molecule_to_graph(raw.smiles, raw.y) for raw in valid_dataset]
# test_features = [molecule_to_graph(raw.smiles) for raw in test_dataset]

In [ ]:
train_loader = DataLoader(train_features, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valid_features, batch_size=batch_size, shuffle=False)

In [ ]:
model = GCN(100, 15, 12)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss(reduction = "none")

## Skip layers

The code was modified with various skip combinations. One of the examples is below. The scores were compared to find if skipping was effective or not.

In [ ]:
# Atom encoder

class AtomEncoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(AtomEncoder, self).__init__()

        self.embeddings = torch.nn.ModuleList()

        for i in range(9):
            self.embeddings.append(torch.nn.Embedding(300, hidden_channels))

    def reset_parameters(self):
        for embedding in self.embeddings:
            embedding.reset_parameters()

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(1)

        out = 0
        for i in range(x.size(1)):
            out += self.embeddings[i](x[:, i])
        return out


# A simple graph neural network model

from torch_geometric.nn import GCNConv, GATv2Conv, DeepGCNLayer
from torch_geometric.nn import global_mean_pool as gap
import torch.nn.functional as F
from torch.nn import Linear
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features, num_classes):
        super(GCN, self).__init__()
        # torch.manual_seed(42)
        self.emb = AtomEncoder(hidden_channels=100)
        self.gat1 = GATv2Conv(hidden_channels, hidden_channels)
        self.gat2 = GATv2Conv(hidden_channels, hidden_channels)
        self.gat3 = GATv2Conv(hidden_channels, hidden_channels)
        self.gat4 = GATv2Conv(hidden_channels, hidden_channels)
        self.gat5 = GATv2Conv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, batch):
        x , edge_index, batch_size = batch.x, batch.edge_index, batch.batch
        x = self.emb(x)
        x = self.gat1(x, edge_index)
        x = x.relu()
        x_out1 = x.clone()
        x = self.gat2(x, edge_index)
        x = x.relu()
        x = self.gat3(x, edge_index)
        x = x.relu()
        x = x + self.gat4(x, edge_index)
        x = x.relu()
        x = x + x_out1
        x = x + self.gat5(x, edge_index)
        

        # 2. Readout layer
        x = gap(x, batch_size)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x